# negotiating the past

To install the environment, see README.md

We load the necessary libraries.

In [8]:
!python --version
%pip install umap-learn
%pip install gensim
%pip install nltk
%pip install pandas

Python 3.11.11


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Creating a Historical Prompt Dataset

In [9]:
# Loading the libraries
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import ssl

# Make sure you have the necessary NLTK resources

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/frederic.clavert/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/frederic.clavert/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### loading the original dataset

In [10]:
# si l'on veut un échantillon aléatoire.
# Définir la taille de l'échantillon souhaité
sample_size = 1000000
# Déterminer le nombre total de lignes (pour les fichiers très grands)
total_rows = sum(1 for _ in open('data/prompts.csv')) - 1  # -1 pour l'en-tête

# Calculer la probabilité de sélection pour chaque ligne
skip_prob = 1 - sample_size / total_rows

# Utiliser le paramètre skiprows avec une fonction lambda
prompts_df = pd.read_csv('data/prompts.csv', 
                        usecols=[0],  # Sélectionne uniquement la première colonne
                        skiprows=lambda x: x > 0 and np.random.random() < skip_prob)

# Si l'on veut juste charger tout le fichier -- attention 10 millions de lignes
# prompts_df = pd.read_csv('data/prompts.csv', usecols=[0])

# on vérifie ce que l'on a engendré:
prompts_df.shape
prompts_df.head(10)

,prompt
0,portrait of an anime girl in white and golden ...
1,"girl with super long hair, hair becoming autum..."
2,"scifi art by greg rutkowski, a very tall, and ..."
3,skinny nerdy poindexter white kid with glasses...
4,"a city made entirely out of Rubik\'s cubes, da..."
5,ancient greek statues taking selfies and watch...
6,what if we will never meet again detailed anal...
7,the batmobile caught in the flow of time. octa...
8,screenshot of a smiling shirtless muscular sen...
9,"short kid wearing a supreme shirt, detailed, s..."


### Historical Reference Detection Approach With DistilBERT

#### Set up the pipeline

In [11]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from umap.umap_ import UMAP
from sklearn.cluster import DBSCAN
from tqdm import tqdm  # Pour les barres de progression
import gc  # Nettoyage de mémoire
import os
from sentence_transformers import SentenceTransformer

# Définir le nombre de cœurs CPU à utiliser
num_cpu_cores = 8
torch.set_num_threads(num_cpu_cores)
os.environ["OMP_NUM_THREADS"] = str(num_cpu_cores)
os.environ["MKL_NUM_THREADS"] = str(num_cpu_cores)

def improved_historical_identification(prompts_df):
    """
    Version améliorée de l'identification des prompts historiques
    """
    # Échantillonnage si nécessaire
    #if len(prompts_df) > sample_size:
    #    sample_df = prompts_df.sample(sample_size, random_state=42)
    #    print(f"Échantillonnage de {sample_size} prompts sur {len(prompts_df)} au total")
    #else:
    #    sample_df = prompts_df.copy()
    #    print(f"Traitement de tous les {len(sample_df)} prompts")


    # Liste des prompts avec filtrage des valeurs non valides
    prompts_list = prompts_df['prompt'].fillna('').tolist()
    valid_text_mask = [isinstance(text, str) and bool(text.strip()) for text in prompts_list]
    valid_prompts = [text for i, text in enumerate(prompts_list) if valid_text_mask[i]]

    print(f"Prompts valides : {len(valid_prompts)} sur {len(prompts_list)}")

    # Filtrer le DataFrame pour ne conserver que les lignes avec texte valide
    prompts_df = prompts_df[valid_text_mask].reset_index(drop=True)

    # Génération des embeddings
    print("Génération des embeddings des prompts...")
    print("Chargement du modèle SentenceTransformer...")
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    # Traitement par lots pour économiser la mémoire
    batch_size = 32
    prompt_embeddings = []

    for i in tqdm(range(0, len(valid_prompts), batch_size), desc="Génération des embeddings"):
        batch = valid_prompts[i:i+batch_size]
        batch_embeddings = model.encode(batch, show_progress_bar=False)
        prompt_embeddings.append(batch_embeddings)

    prompt_embeddings = np.vstack(prompt_embeddings)

    # Vérifier que la longueur correspond
    assert len(prompts_df) == prompt_embeddings.shape[0], "Incohérence entre la longueur du DataFrame et des embeddings"

    # Définir des concepts historiques plus diversifiés
    historical_concepts = [
        # on va s'intéresser aux guerres napoléoniennes
        "napoleon", "napoléon", "Napoléon", "Napoleon", "Bonaparte", "bonaparte", "napoleonic", "Napoleonic", "lest we forget"
        # Périodes historiques explicites
        # "ancient history", "medieval times", "renaissance period",
        # "world war", "cold war", "industrial revolution", "prehistoric era",

        # Termes plus généraux et implicites
        # "history", "historical event", "in the past", "ancient times",
        # "vintage", "retro", "old fashioned", "traditional",

        # Styles et esthétiques historiques
        # "victorian style", "art deco", "baroque", "gothic", "classical",
        # "midcentury", "ancient greek", "roman empire", "medieval knight",

        # Références à la culture populaire historique
        # "steampunk", "dieselpunk", "historical fiction", "period drama",
        # "historical costume", "historical setting"
    ]

    # Générer des embeddings pour les concepts historiques
    print("Génération des embeddings de concepts historiques...")
    historical_embeddings = model.encode(historical_concepts)

    # Calculer la similarité pour chaque concept
    print("Calcul des similarités...")
    concept_similarities = np.zeros((len(valid_prompts), len(historical_concepts)))

    for i in tqdm(range(0, len(valid_prompts), batch_size), desc="Calcul par lots"):
        end_idx = min(i + batch_size, len(valid_prompts))
        batch_embeddings = prompt_embeddings[i:end_idx]

        for j in range(len(historical_concepts)):
            concept_similarities[i:end_idx, j] = cosine_similarity(
                batch_embeddings,
                historical_embeddings[j].reshape(1, -1)
            ).flatten()

    # Pour chaque prompt, trouver le concept le plus similaire
    max_similarity_indices = np.argmax(concept_similarities, axis=1)
    max_similarities = np.max(concept_similarities, axis=1)

    # Attribuer des seuils différents selon les catégories de concepts
    concept_thresholds = {
        # Seuils plus élevés pour termes explicites
        "lest we forget": 0.5,
        "napoleon": 0.5,
        "napoléon": 0.5,
        "Napoléon": 0.5,
        "Napoleon": 0.5,
        "Bonaparte": 0.5,
        "bonaparte": 0.5,
        "napoleonic": 0.5,
        "Napoleonic": 0.5
        #"ancient history": 0.5,
        #"medieval times": 0.5,
        #"renaissance period": 0.5,
        #"world war": 0.5,
        #"cold war": 0.5,
        #"industrial revolution": 0.5,

        # Seuils plus bas pour termes implicites ou généraux
        #"history": 0.45,
        #"historical event": 0.45,
        #"in the past": 0.4,
        #"ancient times": 0.45,

        # Seuils intermédiaires pour styles
        #"victorian style": 0.48,
        #"art deco": 0.48,
        #"baroque": 0.48,
    }

    # Seuil par défaut pour les concepts non listés spécifiquement
    default_threshold = 0.45

    # Vérifier si chaque prompt dépasse le seuil pour son concept le plus similaire
    is_historical = []
    for i, (max_sim, concept_idx) in enumerate(zip(max_similarities, max_similarity_indices)):
        concept = historical_concepts[concept_idx]
        threshold = concept_thresholds.get(concept, default_threshold)
        is_historical.append(max_sim > threshold)

    # Convertir en array numpy pour faciliter l'indexation
    is_historical = np.array(is_historical)

    # Ajouter les résultats au DataFrame
    prompts_df['most_similar_concept'] = [historical_concepts[idx] for idx in max_similarity_indices]
    prompts_df['similarity_score'] = max_similarities
    prompts_df['is_historical'] = is_historical

    # Filtrer les prompts historiques
    historical_prompts = prompts_df[is_historical].copy()

    print(f"Identification de {len(historical_prompts)} prompts historiques ({len(historical_prompts)/len(prompts_df):.2%} de l'échantillon)")

    # Méthode hybride - combiner embeddings et mots-clés pour améliorer la couverture
    keywords = [
         "napoleon", "napoléon", "Napoléon", "Napoleon", "Bonaparte", "bonaparte", "napoleonic", "Napoleonic", "lest we forget"
#        "history", "ancient", "medieval", "renaissance", "century",
#        "war", "empire", "kingdom", "historical", "vintage", "retro",
#        "traditional", "old", "classical", "antique"
    ]

    keyword_mask = prompts_df['prompt'].fillna('').str.lower().str.contains('|'.join(keywords))
    missed_by_embedding = prompts_df[~is_historical & keyword_mask].copy()
    missed_by_embedding['detection_method'] = 'keyword_only'

    # Ajouter un marqueur pour les prompts détectés par embedding
    historical_prompts['detection_method'] = 'embedding'

    # Fusionner les deux ensembles
    combined_historical = pd.concat([historical_prompts, missed_by_embedding], ignore_index=True)

    print(f"Identification supplémentaire de {len(missed_by_embedding)} prompts par mots-clés")
    print(f"Total de prompts historiques: {len(combined_historical)} ({len(combined_historical)/len(prompts_df):.2%} de l'échantillon)")

    return combined_historical, prompt_embeddings, is_historical

def analyze_historical_clusters(historical_prompts, prompt_embeddings, is_historical):
    """
    Version adaptée pour fonctionner avec la nouvelle méthode d'identification
    """
    if len(historical_prompts) > 10:  # Besoin d'un minimum d'échantillons pour le clustering
        # Sélectionner les embeddings des prompts historiques identifiés par embedding
        embedding_historical = historical_prompts[historical_prompts['detection_method'] == 'embedding']

        # Créer un masque pour extraire les embeddings appropriés
        # Note: Cette approche suppose que les indices des historical_prompts correspondent
        # aux indices filtrés dans prompt_embeddings
        historical_indices = embedding_historical.index
        historical_embeddings = prompt_embeddings[historical_indices]

        print("Application de la réduction dimensionnelle UMAP...")
        # Appliquer UMAP avec optimisation CPU
        umap_model = UMAP(n_neighbors=15, min_dist=0.1, random_state=42,
                          n_jobs=num_cpu_cores)
        historical_umap = umap_model.fit_transform(historical_embeddings)

        print("Clustering avec DBSCAN...")
        # Appliquer DBSCAN
        dbscan = DBSCAN(eps=0.5, min_samples=5, n_jobs=num_cpu_cores)
        cluster_labels = dbscan.fit_predict(historical_umap)

        # Ajouter les étiquettes de cluster au DataFrame, uniquement pour les prompts détectés par embedding
        # Initialiser une colonne de cluster avec des valeurs NaN
        historical_prompts['cluster'] = float('nan')

        # Mettre à jour les valeurs de cluster pour les prompts détectés par embedding
        idx_map = {old_idx: new_idx for new_idx, old_idx in enumerate(historical_indices)}
        for i, idx in enumerate(historical_indices):
            historical_prompts.loc[idx, 'cluster'] = cluster_labels[i]

        # Générer rapport et visualisations, uniquement pour les prompts avec clusters
        generate_cluster_report(embedding_historical, historical_umap, cluster_labels)

        return historical_prompts, historical_umap, cluster_labels
    else:
        print("Pas assez de prompts historiques pour le clustering")
        return historical_prompts, None, None

def generate_cluster_report(historical_prompts, historical_umap, cluster_labels):
    # Nombre de prompts par cluster
    cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
    print("\nNombre de prompts par cluster:")
    print(cluster_counts)

    # Visualiser les clusters
    plt.figure(figsize=(12, 10))
    # Exclure les points de bruit (cluster -1) pour une meilleure visualisation
    non_noise_mask = cluster_labels != -1

    if np.any(non_noise_mask):  # Vérifier qu'il y a des points non-bruit
        scatter = plt.scatter(
            historical_umap[non_noise_mask, 0],
            historical_umap[non_noise_mask, 1],
            c=cluster_labels[non_noise_mask],
            cmap='tab20',
            alpha=0.6,
            s=10
        )
        plt.colorbar(scatter, label='Cluster')
    else:
        # S'il n'y a que des points de bruit, tous les afficher
        plt.scatter(
            historical_umap[:, 0],
            historical_umap[:, 1],
            c='gray',
            alpha=0.3,
            s=5
        )

    plt.title('Clusters des Prompts Historiques (méthode améliorée)')
    plt.xlabel('UMAP Dimension 1')
    plt.ylabel('UMAP Dimension 2')
    plt.savefig('historical_prompt_clusters_improved.png', dpi=300, bbox_inches='tight')
#    plt.close()  # Fermer la figure pour libérer la mémoire

    # Exemples de prompts de chaque cluster
    print("\nExemples de prompts par cluster:")
    for cluster_id in sorted(set([label for label in cluster_labels if label != -1])):
        # Créer un masque pour ce cluster
        cluster_mask = cluster_labels == cluster_id

        # Trouver les indices correspondants dans historical_prompts
        cluster_indices = np.where(cluster_mask)[0]

        # Récupérer les lignes correspondantes
        if len(cluster_indices) > 0:
            # Échantillonner jusqu'à 5 prompts de ce cluster
            sample_indices = np.random.choice(cluster_indices, min(5, len(cluster_indices)), replace=False)

            print(f"\nCluster {cluster_id} ({len(cluster_indices)} prompts):")

            for idx in sample_indices:
                row_idx = historical_prompts.index[idx]
                row = historical_prompts.loc[row_idx]
                print(f"  - {row['prompt'][:100]}... (Similarité: {row['similarity_score']:.2f}, Concept: {row['most_similar_concept']})")

# Fonction principale pour exécuter l'ensemble du pipeline
def run_historical_prompt_analysis(prompts_df):
    # Identification des prompts historiques avec la méthode améliorée
    historical_prompts, prompt_embeddings, is_historical = improved_historical_identification(prompts_df)

    # Analyse de clusters sur les prompts historiques identifiés
    historical_prompts_clustered, umap_result, cluster_labels = analyze_historical_clusters(
        historical_prompts, prompt_embeddings, is_historical)

    return historical_prompts_clustered, umap_result, cluster_labels


#### we export the clusters

In [12]:
# we also export the CSVs

def export_clusters_to_csv(historical_prompts, output_dir="data/generated/cluster_exports"):
    """
    Exporte les prompts regroupés par cluster dans des fichiers CSV distincts.
    Chaque fichier contient les prompts, scores de similarité et concepts associés.

    Parameters:
    -----------
    historical_prompts : pandas.DataFrame
        DataFrame contenant les prompts historiques avec les colonnes:
        - prompt: texte du prompt
        - similarity_score: score de similarité
        - most_similar_concept: concept historique le plus similaire
        - cluster: étiquette de cluster (-1 pour les points de bruit)
        - detection_method: méthode ayant détecté le prompt (embedding ou keyword_only)

    output_dir : str
        Répertoire de sortie pour les fichiers CSV
    """
    # Créer le répertoire de sortie s'il n'existe pas
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Répertoire créé: {output_dir}")

    # Extraire les prompts avec clusters valides (détection par embedding)
    clustered_prompts = historical_prompts[historical_prompts['cluster'].notna()].copy()

    # Ajouter une version tronquée du prompt pour la prévisualisation
    clustered_prompts['prompt_preview'] = clustered_prompts['prompt'].apply(
        lambda x: x[:100] + '...' if len(x) > 100 else x
    )

    # Créer un fichier récapitulatif pour tous les clusters
    summary_file = os.path.join(output_dir, "all_clusters_summary.csv")
    clusters_summary = clustered_prompts.groupby('cluster').agg(
        prompt_count=('prompt', 'count'),
        avg_similarity=('similarity_score', 'mean'),
        top_concepts=('most_similar_concept', lambda x: x.value_counts().index[0] if len(x) > 0 else 'N/A'),
        min_similarity=('similarity_score', 'min'),
        max_similarity=('similarity_score', 'max')
    ).reset_index()

    # Ajouter des exemples de prompts au résumé (3 exemples par cluster)
    def get_sample_prompts(group):
        if len(group) <= 3:
            return '; '.join(group['prompt_preview'].tolist())
        else:
            return '; '.join(group.sample(3)['prompt_preview'].tolist())

    sample_prompts = clustered_prompts.groupby('cluster').apply(get_sample_prompts).reset_index()
    sample_prompts.columns = ['cluster', 'sample_prompts']
    clusters_summary = pd.merge(clusters_summary, sample_prompts, on='cluster')

    # Enregistrer le résumé
    clusters_summary.to_csv(summary_file, index=False, encoding='utf-8')
    print(f"Résumé des clusters enregistré dans {summary_file}")

    # Exporter chaque cluster vers un fichier CSV distinct
    unique_clusters = clustered_prompts['cluster'].unique()

    for cluster_id in unique_clusters:
        # Ignorer les points de bruit pour l'exportation individuelle
        if cluster_id == -1:
            noise_file = os.path.join(output_dir, "noise_points.csv")
            noise_prompts = clustered_prompts[clustered_prompts['cluster'] == -1]
            if not noise_prompts.empty:
                noise_prompts.to_csv(noise_file, index=False, encoding='utf-8')
                print(f"Points de bruit ({len(noise_prompts)}) enregistrés dans {noise_file}")
            continue

        # Filtrer les prompts de ce cluster
        cluster_df = clustered_prompts[clustered_prompts['cluster'] == cluster_id].copy()

        # Trier par similarité décroissante
        cluster_df = cluster_df.sort_values('similarity_score', ascending=False)

        # Analyser les concepts présents dans ce cluster
        concept_counts = cluster_df['most_similar_concept'].value_counts()
        top_concepts = concept_counts.head(3).to_dict()  # Top 3 concepts

        # Créer un fichier avec un nom informatif
        top_concept = concept_counts.index[0] if len(concept_counts) > 0 else "inconnu"
        top_concept_safe = ''.join(c if c.isalnum() else '_' for c in top_concept)  # Nom de fichier sécurisé

        file_name = f"cluster_{int(cluster_id):02d}_{top_concept_safe}_{len(cluster_df)}_prompts.csv"
        file_path = os.path.join(output_dir, file_name)

        # Enregistrer ce cluster en CSV
        cluster_df.to_csv(file_path, index=False, encoding='utf-8')
        print(f"Cluster {int(cluster_id)} ({top_concept}, {len(cluster_df)} prompts) enregistré dans {file_name}")

    # Exporter les prompts détectés uniquement par mots-clés
    keyword_only = historical_prompts[historical_prompts['detection_method'] == 'keyword_only']
    if not keyword_only.empty:
        keyword_file = os.path.join(output_dir, "keyword_only_prompts.csv")
        keyword_only.to_csv(keyword_file, index=False, encoding='utf-8')
        print(f"Prompts détectés par mots-clés uniquement ({len(keyword_only)}) enregistrés dans {keyword_file}")

    # Créer un fichier HTML pour explorer les clusters
    create_html_explorer(historical_prompts, os.path.join(output_dir, "cluster_explorer.html"))

    return clusters_summary

def create_html_explorer(historical_prompts, output_file):
    """
    Crée un fichier HTML simple pour explorer les clusters de prompts.

    Parameters:
    -----------
    historical_prompts : pandas.DataFrame
        DataFrame contenant les prompts historiques avec clustering
    output_file : str
        Chemin vers le fichier HTML de sortie
    """
    # Extraire uniquement les prompts avec clusters valides
    clustered_prompts = historical_prompts[historical_prompts['cluster'].notna()].copy()

    # Récupérer la liste des clusters uniques (triés)
    unique_clusters = sorted(clustered_prompts['cluster'].unique())

    # Calculer des statistiques pour chaque cluster
    cluster_stats = {}
    for cluster_id in unique_clusters:
        if cluster_id == -1:  # Traiter les points de bruit séparément
            continue

        cluster_df = clustered_prompts[clustered_prompts['cluster'] == cluster_id]
        concept_counts = cluster_df['most_similar_concept'].value_counts().head(3)

        cluster_stats[cluster_id] = {
            'size': len(cluster_df),
            'avg_similarity': cluster_df['similarity_score'].mean(),
            'top_concepts': dict(concept_counts),
            'samples': cluster_df.sort_values('similarity_score', ascending=False).head(5)['prompt'].tolist()
        }

    # Générer le HTML
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Explorateur de Clusters de Prompts Historiques</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; line-height: 1.6; }
            h1 { color: #2c3e50; }
            h2 { color: #3498db; margin-top: 30px; }
            .cluster { border: 1px solid #ddd; padding: 15px; margin: 15px 0; border-radius: 5px; }
            .cluster-header { display: flex; justify-content: space-between; }
            .concept-tag { display: inline-block; background: #e0f7fa; padding: 3px 8px; margin: 3px; border-radius: 3px; }
            .prompt-item { padding: 8px; margin: 5px 0; background: #f9f9f9; border-left: 3px solid #3498db; }
            .similarity { font-size: 0.8em; color: #666; }
            .stats { color: #7f8c8d; font-size: 0.9em; }
            .toggle-button { background: #3498db; color: white; border: none; padding: 5px 10px; cursor: pointer; border-radius: 3px; }
            .prompt-list { max-height: 0; overflow: hidden; transition: max-height 0.3s ease-out; }
            .expanded { max-height: 2000px; }
            .search-container { margin: 20px 0; }
            #searchInput { padding: 8px; width: 250px; }
            #statsSection { margin-top: 30px; }
        </style>
    </head>
    <body>
        <h1>Explorateur de Clusters de Prompts Historiques</h1>

        <div class="search-container">
            <input type="text" id="searchInput" placeholder="Rechercher des prompts...">
            <button onclick="searchPrompts()">Rechercher</button>
        </div>

        <div id="statsSection">
            <h2>Statistiques générales</h2>
            <p>Nombre total de clusters: <strong>""" + str(len(cluster_stats)) + """</strong></p>
            <p>Nombre total de prompts clusterisés: <strong>""" + str(len(clustered_prompts)) + """</strong></p>
        </div>

        <div id="clustersSection">
    """

    # Ajouter chaque cluster au HTML
    for cluster_id in sorted([c for c in unique_clusters if c != -1]):
        stats = cluster_stats[cluster_id]

        # Créer les balises de concepts
        concept_tags = ""
        for concept, count in stats['top_concepts'].items():
            concept_tags += f'<span class="concept-tag">{concept} ({count})</span>'

        # Ajouter les prompts échantillons
        prompt_items = ""
        for i, prompt in enumerate(stats['samples']):
            # Raccourcir le prompt pour l'affichage
            display_prompt = prompt[:200] + "..." if len(prompt) > 200 else prompt
            prompt_items += f'<div class="prompt-item">{i+1}. {display_prompt}</div>'

        # Créer la section de cluster
        html_content += f"""
        <div class="cluster" data-cluster-id="{int(cluster_id)}">
            <div class="cluster-header">
                <h3>Cluster {int(cluster_id)}</h3>
                <button class="toggle-button" onclick="togglePrompts(this)">Afficher les prompts</button>
            </div>
            <div class="stats">
                <p><strong>Taille:</strong> {stats['size']} prompts | <strong>Similarité moyenne:</strong> {stats['avg_similarity']:.2f}</p>
                <p><strong>Concepts principaux:</strong> {concept_tags}</p>
            </div>
            <div class="prompt-list">
                <h4>Exemples de prompts:</h4>
                {prompt_items}
            </div>
        </div>
        """

    # Ajouter les points de bruit
    noise_df = clustered_prompts[clustered_prompts['cluster'] == -1]
    if len(noise_df) > 0:
        noise_samples = noise_df.sample(min(5, len(noise_df)))['prompt'].tolist()
        noise_items = ""
        for i, prompt in enumerate(noise_samples):
            display_prompt = prompt[:200] + "..." if len(prompt) > 200 else prompt
            noise_items += f'<div class="prompt-item">{i+1}. {display_prompt}</div>'

        html_content += f"""
        <div class="cluster">
            <div class="cluster-header">
                <h3>Points de bruit</h3>
                <button class="toggle-button" onclick="togglePrompts(this)">Afficher les prompts</button>
            </div>
            <div class="stats">
                <p><strong>Taille:</strong> {len(noise_df)} prompts</p>
            </div>
            <div class="prompt-list">
                <h4>Exemples de prompts:</h4>
                {noise_items}
            </div>
        </div>
        """

    # Ajouter le JavaScript et fermer les balises HTML
    html_content += """
        </div>

        <script>
            function togglePrompts(button) {
                const list = button.parentNode.nextElementSibling.nextElementSibling;
                list.classList.toggle('expanded');
                button.textContent = list.classList.contains('expanded') ? 'Masquer les prompts' : 'Afficher les prompts';
            }

            function searchPrompts() {
                const searchTerm = document.getElementById('searchInput').value.toLowerCase();
                const clusters = document.querySelectorAll('.cluster');

                if (searchTerm === '') {
                    // Si la recherche est vide, afficher tous les clusters
                    clusters.forEach(cluster => {
                        cluster.style.display = 'block';
                    });
                    return;
                }

                clusters.forEach(cluster => {
                    const promptItems = cluster.querySelectorAll('.prompt-item');
                    let matchFound = false;

                    promptItems.forEach(item => {
                        if (item.textContent.toLowerCase().includes(searchTerm)) {
                            matchFound = true;
                        }
                    });

                    // Aussi vérifier dans les concepts
                    const conceptTags = cluster.querySelectorAll('.concept-tag');
                    conceptTags.forEach(tag => {
                        if (tag.textContent.toLowerCase().includes(searchTerm)) {
                            matchFound = true;
                        }
                    });

                    cluster.style.display = matchFound ? 'block' : 'none';
                });
            }
        </script>
    </body>
    </html>
    """

    # Écrire le HTML dans un fichier
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"Explorateur HTML créé: {output_file}")



### Run the pipline

In [13]:
# Exécuter l'analyse et utiliser directement le résultat
results, _, _ = run_historical_prompt_analysis(prompts_df)

# Exporter avec les résultats
summary = export_clusters_to_csv(results)

Prompts valides : 988927 sur 998862
Génération des embeddings des prompts...
Chargement du modèle SentenceTransformer...


Génération des embeddings:  33%|███▎      | 10304/30904 [2:00:17<4:14:30,  1.35it/s] 

In [ ]:
# et l'on regarde ce qu'il y a en mémoire
%whos DataFrame